In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
pd.options.display.max_columns = 50
plt.rcParams["figure.figsize"] = (20,12)

In [222]:
train = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\train.csv")
test = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\test.csv")

In [223]:
alldata = pd.concat([train,test])
# alldata = alldata.drop(alldata[alldata["dered_g"] == -9999].index)
# alldata = alldata.drop(alldata[alldata["dered_i"] == -9999].index)
# alldata = alldata.drop(alldata[alldata["dered_z"] == -9999].index)
# alldata = alldata.drop(alldata[alldata["z"] == -9999].index)
alldata.describe()

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,320000.000000
mean,199999.500000,19.853911,18.388891,17.736557,17.200679,16.929940,0.058663,19.630089,18.234426,17.616215,17.165439,16.869468,6.336150,6.142602,1.175802,1.176569,1.175167,1.175467,1.176168,1.116322
std,115470.198175,1.947292,27.836731,1.468719,47.016128,52.287872,0.537570,1.923074,15.926223,1.473616,22.436363,35.436536,8.879454,8.551472,0.116323,0.118092,0.114748,0.115510,0.117182,0.923455
min,0.000000,-17.687512,-17555.365606,-5.438650,-23955.817764,-23955.817764,-166.048260,-42.837530,-9999.000000,-16.380230,-9999.000000,-9999.000000,1.000000,1.000000,1.000059,1.000012,1.000157,1.000105,1.000027,0.000000
25%,99999.750000,18.724761,17.475412,16.878248,16.524128,16.288817,0.000035,18.565090,17.349170,16.787888,16.453717,16.233397,1.000000,1.000000,1.088191,1.088428,1.087747,1.087998,1.088278,0.000000
50%,199999.500000,19.420835,18.141361,17.526730,17.151000,16.918373,0.047127,19.265195,18.022860,17.444390,17.088090,16.870060,2.000000,2.000000,1.179350,1.179242,1.179375,1.179381,1.179306,1.000000
75%,299999.250000,20.432632,19.074540,18.428941,18.007443,17.729659,0.094629,20.198145,18.890170,18.292180,17.907530,17.655963,5.000000,5.000000,1.227600,1.226032,1.229272,1.228639,1.226807,2.000000
max,399999.000000,58.623328,94.359125,72.009662,50.164052,6976.392220,62.323343,30.889910,30.952920,31.957200,32.016220,30.781270,44.000000,42.000000,2.050775,2.080287,2.022134,2.036329,2.065390,2.000000


In [224]:
# alldata[alldata["dered_g"] == -9999] # 1
# alldata[alldata["dered_i"] == -9999] # 2
# alldata[alldata["dered_z"] == -9999] # 2
# alldata[alldata["dered_z"] == -9999] # 2

In [225]:
# plt.figure(figsize = (20,12))
# sns.heatmap(train.corr(), annot = True, annot_kws = {"size" : 11}, linewidths = 1 )

망원경에서 망원경으로 또는 밤에서 밤까지 절대교정과 관련된 불확실성을 줄이기 위해 

천문학자들은 일반적으로 두 개의 서로 다른 필터 밴드 사이의 크기 차이로 정의되는 색상으로 작업한다. 

두 개의 크기를 빼면 이러한 불확실성이 감소한다. 

예를 들어, 위의 별 베가에 대한 관측은 벡터 4개의 숫자로 구성된다. [u-g, g-r, r-i, i-z]

In [226]:
alldata["1-color"] = -0.436 * alldata["u"] + 1.129 * alldata["g"] - 0.119 * alldata["r"] - 0.574 * alldata["i"] + 0.1984
alldata["s-color"] = -0.249 * alldata["u"] + 0.794 * alldata["g"] - 0.555 * alldata["r"] + 0.234
alldata["p1s"] = 0.91 * (alldata["u"] - alldata["g"]) + 0.415 * (alldata["g"] - alldata["r"]) - 1.280
alldata["c1"] = (alldata["r"] - alldata["i"]) - (alldata["g"] - alldata["r"]) / 4 - 0.177
alldata["c2"] = 0.7 * (alldata["g"] - alldata["r"]) + 1.2 * (alldata["r"] - alldata["i"] - 0.177)

In [227]:
alldata["u-g"] = alldata["u"]-alldata["g"]
alldata["g-r"] = alldata["g"]-alldata["r"]
alldata["r-i"] = alldata["r"]-alldata["i"]
alldata["i-z"] = alldata["i"]-alldata["z"]

alldata["dered_diff1"] = alldata["dered_u"]-alldata["dered_g"]
alldata["dered_diff2"] = alldata["dered_g"]-alldata["dered_r"]
alldata["dered_diff3"] = alldata["dered_r"]-alldata["dered_i"]
alldata["dered_diff4"] = alldata["dered_i"]-alldata["dered_z"]

alldata["minusOD"] = alldata["nObserve"] - alldata["nDetect"]   # 0.1560

# AB (absolute bolometric) magnitude
# alldata["1"] = (-2.5)*np.log1p(alldata["u"]) - 48.60
# alldata["2"] = (-2.5)*np.log1p(alldata["g"]) - 48.60
# alldata["3"] = (-2.5)*np.log1p(alldata["r"]) - 48.60
# alldata["4"] = (-2.5)*np.log1p(alldata["i"]) - 48.60
# alldata["5"] = (-2.5)*np.log1p(alldata["z"]) - 48.60

# alldata["6"] = np.log1p(alldata["u"])
# alldata["7"] = np.log1p(alldata["g"])
# alldata["8"] = np.log1p(alldata["r"])
# alldata["9"] = np.log1p(alldata["i"])
# alldata["10"] = np.log1p(alldata["z"])
    
alldata["d5"] = alldata["u"]-alldata["r"] # 6591  
alldata["d6"] = alldata["u"]-alldata["i"] # 6648   
alldata["d7"] = alldata["u"]-alldata["z"]  # 6548   
alldata["d8"] = alldata["g"]-alldata["i"] # 6639   
alldata["d9"] = alldata["g"]-alldata["z"] # 6608   
alldata["d10"] = alldata["r"]-alldata["z"] # 6614   
alldata['d11'] = alldata['dered_u'] - alldata['dered_z'] # 6595
alldata['d_dered_ig'] = alldata['dered_i'] - alldata['dered_g'] # 6605
alldata['d_dered_zg'] = alldata['dered_z'] - alldata['dered_g'] # 6641
alldata['d_dered_rz'] = alldata['dered_r'] - alldata['dered_z'] # 6641

alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,1-color,s-color,p1s,c1,c2,u-g,g-r,r-i,i-z,dered_diff1,dered_diff2,dered_diff3,dered_diff4,minusOD,d5,d6,d7,d8,d9,d10,d11,d_dered_ig,d_dered_zg,d_dered_rz
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0,0.609422,0.038400,1.934555,0.828243,2.321164,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317,0,4.254456,5.591517,6.324349,2.664334,3.397167,2.069894,6.21532,-2.62568,-3.34885,2.04618
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0,0.294699,0.185126,-0.125974,-0.145419,0.435048,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157,0,1.599729,1.783698,1.883207,0.793519,0.893028,0.283477,1.83174,-0.76827,-0.86984,0.27671
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0,0.061731,-0.003078,-0.285548,-0.106494,0.161315,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565,0,1.250067,1.392850,1.436353,0.431891,0.475394,0.186286,1.31157,-0.36486,-0.39051,0.14396
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0,-0.893584,-0.554392,3.190980,-0.092683,0.856349,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947,1,5.439479,5.765689,6.025989,1.293778,1.554078,0.586509,5.79847,-1.20000,-1.43947,0.53209
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0,0.121963,0.011723,2.823949,0.127015,1.809220,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436,1,5.411062,6.129277,6.470780,2.375018,2.716521,1.059719,5.89548,-2.24356,-2.55792,0.98707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN,0.113279,0.012398,-0.197116,-0.126431,0.239156,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911,0,1.402600,1.550887,1.610717,0.539161,0.598990,0.208117,1.36915,-0.40721,-0.42632,0.11901
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN,-0.292574,-0.104765,-0.507297,-0.603076,-0.604105,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801,4,0.914174,0.517994,0.677070,-0.276593,-0.117518,-0.237104,0.61667,0.31077,0.14276,-0.23762
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN,-0.272026,-0.192026,-0.192788,-0.234408,-0.200705,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574,0,1.238573,1.201312,1.197234,0.043323,0.039245,-0.041340,0.87359,0.12893,0.16467,-0.13531
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN,0.134651,-0.005695,0.096196,0.016533,0.487831,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745,0,1.766869,2.077400,2.305186,0.778522,1.006307,0.538316,2.21303,-0.72203,-0.92948,0.49717


In [228]:
alldata2 = alldata.drop(["id","class","airmass_u","airmass_g","airmass_r","airmass_i","airmass_z"], axis=1)
alldata2

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,1-color,s-color,p1s,c1,c2,u-g,g-r,r-i,i-z,dered_diff1,dered_diff2,dered_diff3,dered_diff4,minusOD,d5,d6,d7,d8,d9,d10,d11,d_dered_ig,d_dered_zg,d_dered_rz
0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,0.609422,0.038400,1.934555,0.828243,2.321164,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317,0,4.254456,5.591517,6.324349,2.664334,3.397167,2.069894,6.21532,-2.62568,-3.34885,2.04618
1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,0.294699,0.185126,-0.125974,-0.145419,0.435048,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157,0,1.599729,1.783698,1.883207,0.793519,0.893028,0.283477,1.83174,-0.76827,-0.86984,0.27671
2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,0.061731,-0.003078,-0.285548,-0.106494,0.161315,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565,0,1.250067,1.392850,1.436353,0.431891,0.475394,0.186286,1.31157,-0.36486,-0.39051,0.14396
3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,-0.893584,-0.554392,3.190980,-0.092683,0.856349,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947,1,5.439479,5.765689,6.025989,1.293778,1.554078,0.586509,5.79847,-1.20000,-1.43947,0.53209
4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,0.121963,0.011723,2.823949,0.127015,1.809220,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436,1,5.411062,6.129277,6.470780,2.375018,2.716521,1.059719,5.89548,-2.24356,-2.55792,0.98707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,0.113279,0.012398,-0.197116,-0.126431,0.239156,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911,0,1.402600,1.550887,1.610717,0.539161,0.598990,0.208117,1.36915,-0.40721,-0.42632,0.11901
79996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,-0.292574,-0.104765,-0.507297,-0.603076,-0.604105,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801,4,0.914174,0.517994,0.677070,-0.276593,-0.117518,-0.237104,0.61667,0.31077,0.14276,-0.23762
79997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,-0.272026,-0.192026,-0.192788,-0.234408,-0.200705,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574,0,1.238573,1.201312,1.197234,0.043323,0.039245,-0.041340,0.87359,0.12893,0.16467,-0.13531
79998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,0.134651,-0.005695,0.096196,0.016533,0.487831,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745,0,1.766869,2.077400,2.305186,0.778522,1.006307,0.538316,2.21303,-0.72203,-0.92948,0.49717


In [229]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [230]:
x_train, x_valid, y_train, y_valid = train_test_split(train2,train["class"], 
                                                      test_size = 0.2, 
                                                      random_state = 42, 
                                                      shuffle = True, 
                                                      stratify = train["class"])

In [ ]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp

# 초모수 탐색공간 정의
param_space = {'n_estimators' : hp.quniform('n_estimators', 500, 4500, 250),
               'num_leaves': hp.quniform('num_leaves', 120, 210, 1),
               'subsample': hp.uniform('subsample', 0.5, 1),
               'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
               'learning_rate' : hp.quniform('learning_rate',0.01,0.1, 0.01)
              }

# 목적함수 정의
def objective(params):
    params = {'n_estimators' : int(params['n_estimators']),
              'num_leaves': int(params['num_leaves']),
              'subsample': params['subsample'],
              'colsample_bytree' : params['colsample_bytree'],
              'learing_rate' : params['learning_rate']
             }
        
    xgb_clf = LGBMClassifier(**params, n_jobs = -1, sub) 
    best_score = cross_val_score(xgb_clf, train2, train["class"], 
                                 scoring='accuracy', 
                                 cv=2, 
                                 n_jobs=-1).mean()
    
#     xgb_clf.fit(x_train,y_train, eval_set = (x_train,x_valid), early_stopping_rounds = 30)
#     best_score = xgb_clf.score(x_train, x_valid)
    
    loss = 1 - best_score
    return loss


# 알고리즘 실행
best = fmin(fn=objective, space=param_space, 
            max_evals=10, 
            rstate=np.random.RandomState(777), 
            algo=tpe.suggest)
print(best)

  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

In [231]:
# # 베이지안
# from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin

# params = {
#     "objective": "multiclass",
#     "n_estimators": 3000,
#     "subsample_freq": 1,
#     "random_state": 42,
#     "n_jobs": 6,
# }

# space = {
#     "learning_rate": hp.uniform("learning_rate", 0.01, 0.1),
#     "colsample_bytree": hp.uniform("colsample_bytree", .5, 1),
#     "subsample": hp.uniform("subsample", .5, 1),
#     "num_leaves": hp.choice("num_leaves", [110,120,130,140,150,160,170,180,190,210,220]),
#     "min_child_samples": hp.choice('min_child_samples', [10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100])
#     }


# def objective(hyperparams):
#     model = LGBMClassifier(**params, **hyperparams)
#     model.fit(X=x_train, y=y_train,
#               eval_set=[(x_valid, y_valid)],
#               eval_metric="multi_logloss",
#               early_stopping_rounds=80,
#               verbose=False)
#     score = model.best_score_["valid_0"]["multi_logloss"]

#     return {'loss': score, 'status': STATUS_OK, 'model': model}

# trials = Trials()
# best = fmin(fn=objective, space=space, trials=trials,
#             algo=tpe.suggest, max_evals=10, verbose=1)

# hyperparams = space_eval(space, best)
# n_best = trials.best_trial['result']['model'].best_iteration_
# params.update(hyperparams)
# print(params)

In [232]:
# 모델 만들기
#x_train, x_valid, y_train, y_valid = train_test_split(train2,train["class"], 
#                                                       test_size = 0.2, 
#                                                       random_state = 42, 
#                                                       shuffle = True, 
#                                                       stratify = train["class"])

# lgb모델
# 1554 = n_estimators = 1500, num_leaves = 200, n_jobs = -1, learning_rate = 0.01
lgb = LGBMClassifier(n_estimators = 2000, n_jobs = -1, random_state = 42 )
# lgb.fit(x_train,y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 50)

# lgb = LGBMClassifier(objective='multiclass', # 1563
#                      n_estimators = 2000, 
#                      num_leaves=160,
#                      learning_rate=0.01954777244313794,
#                      n_jobs = -1, 
#                      min_child_samples = 20, 
#                      colsample_bytree =  0.794463123335423,
#                      random_state=42,
#                      subsample = 0.6142434932491605, 
#                      subsample_freq = 1)
                     
# lgb = LGBMClassifier(objective= 'multiclass', # 1537.23
#                      n_estimators = 2000, 
#                      subsample_freq= 1, 
#                      random_state= 42, 
#                      n_jobs= 6 
#                     )
#                      colsample_bytree= 0.703, 
#                      learning_rate=  0.046, 
#                      min_child_samples= 30, 
#                      num_leaves= 140, 
#                      subsample= 0.664)


lgb.fit(x_train,y_train, eval_set = (x_valid, y_valid), eval_metric="multiclass" ,early_stopping_rounds = 80)
# # result = lgb.predict_proba(test2)

p_val = lgb.predict(x_valid)
p_tst = lgb.predict(test2)

print(f'{accuracy_score(y_valid, p_val) * 100:.4f}%')

[1]	valid_0's multi_logloss: 0.876948
Training until validation scores don't improve for 80 rounds
[2]	valid_0's multi_logloss: 0.788568
[3]	valid_0's multi_logloss: 0.714757
[4]	valid_0's multi_logloss: 0.652274
[5]	valid_0's multi_logloss: 0.59863
[6]	valid_0's multi_logloss: 0.552255
[7]	valid_0's multi_logloss: 0.511677
[8]	valid_0's multi_logloss: 0.476172
[9]	valid_0's multi_logloss: 0.444899
[10]	valid_0's multi_logloss: 0.417243
[11]	valid_0's multi_logloss: 0.392783
[12]	valid_0's multi_logloss: 0.370967
[13]	valid_0's multi_logloss: 0.351609
[14]	valid_0's multi_logloss: 0.334303
[15]	valid_0's multi_logloss: 0.318792
[16]	valid_0's multi_logloss: 0.304855
[17]	valid_0's multi_logloss: 0.292298
[18]	valid_0's multi_logloss: 0.28099
[19]	valid_0's multi_logloss: 0.270722
[20]	valid_0's multi_logloss: 0.261469
[21]	valid_0's multi_logloss: 0.253044
[22]	valid_0's multi_logloss: 0.245483
[23]	valid_0's multi_logloss: 0.238668
[24]	valid_0's multi_logloss: 0.232419
[25]	valid_0's

93.3250%


In [233]:
93.6078

93.6078

In [234]:
# cv = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
# p_val = np.zeros((train2.shape[0], 3))
# p_tst = np.zeros((test2.shape[0], 3))

# for i, (i_trn, i_val) in enumerate(cv.split(train2, train["class"]), 1):
#     print(f'training model for CV #{i}')
#     clf = LGBMClassifier(objective= 'multiclass', # 1537.23
#                      n_estimators = 1500, 
#                      subsample_freq= 1, 
#                      random_state= 42, 
#                      n_jobs= 6, 
#                      colsample_bytree= 0.589, 
#                      learning_rate=  0.0130, 
#                      min_child_samples= 20, 
#                      num_leaves= 170, 
#                      subsample= 0.828) 
#     clf.fit(train2.iloc[i_trn], train["class"].iloc[i_trn])
    
#     p_val[i_val, :] = clf.predict_proba(train2.iloc[i_val]) # p_val : 스태킹 x_train
#     p_tst += clf.predict_proba(test2) / 10                  # p_tst : 스태킹 x_test
    

# lgb_train_stack = pd.DataFrame(p_val)
# lgb_test_stack = pd.DataFrame(p_tst)
# lgb_train_stack.to_csv("lgb_train_stack.csv", index = False)
# lgb_test_stack.to_csv("lgb_test_stack.csv", index = False)

In [235]:
######  Kfold 제출용 
# sfk = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
# result = 0

# for i,j in sfk.split(train2,train["class"]):
#     x_train = train2.iloc[i]
#     x_valid = train2.iloc[j]
#     y_train = train["class"].iloc[i]
#     y_valid = train["class"].iloc[j]

#     lgb = LGBMClassifier(objective='multiclass',
#                      n_estimators = 4000, 
#                      num_leaves=200,
#                      learning_rate=0.01,
#                      n_jobs = 3, 
#                      min_child_samples = 10, 
#                      colsample_bytree = 0.6,
#                      random_state=42,
#                      subsample = 0.8, 
#                      subsample_freq = 1)
    
#     lgb.fit(x_train,y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 200)
# #     result += lgb.predict_proba(test2)/10
#     lgb_stack += lgb.predict_proba(x_valid)/10
    
    
# lgb_result = pd.DataFrame(result)
# lgb_result.to_csv("lgb_result.csv", index = False)

In [236]:
# print(accuracy_score(np.array(y_valid).astype(int),result.argmax(1)))
# confusion_matrix(np.array(y_valid).astype(int),result.argmax(1)) 

In [237]:
sub = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\sample_submission.csv")
sub

,id,class
0,320000,0
1,320001,0
2,320002,0
3,320003,0
4,320004,0
...,...,...
79995,399995,0
79996,399996,0
79997,399997,0
79998,399998,0


In [238]:
sub["class"] = result.argmax(1)
sub

NameError: name 'result' is not defined

In [ ]:
sub.to_csv("lightgbm과제.csv", index=False)

In [ ]:
# pd.Series(lgb.feature_importances_, index = train2.columns).sort_values(ascending = False).reset_index()

imp = pd.DataFrame({'feature': train2.columns, 'importance': lgb.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh')